# Word2Vec con Gensim

## Instalación de librerías y carga de dataset

In [1]:
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

-------------------------

In [3]:
from datasets import load_dataset

dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl", trust_remote_code=True)

Generating train split: 100%|██████████| 15510649/15510649 [06:32<00:00, 39532.49 examples/s]


In [4]:
dataset_corpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15510649
    })
})

In [38]:
subset = dataset_corpus['train'].select(range(1000000))
subset[1:3]

{'text': ['Sin un constante aluvión de doble cañón, requiriendo la complicidad de los seres humanos para reprimir y engañar a sus semejantes, su tan cacareada magia rápidamente se desvanecería y se disiparía.',
  'En realidad, el Nuevo OM sólo se puede mantener la ilusión de supremacía mágica, siempre y cuando reprima y desvíe el potencial humano, donde mora la verdadera magia: es decir, en la capacidad innata de nuestra especie de magia interactiva con los poderes de animación de la diosa planetaria.']}

In [7]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zer0\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zer0\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

## Procesamiento de texto

In [20]:
STOP_WORDS = set(stopwords.words('spanish'))

URLS_RE = re.compile(r'(https?|www)\S+', flags=re.MULTILINE)
SOCIAL_RE = re.compile(r'[@#]\w+')

def clean_text(sentence_batch):
    # extrae el texto de la entrada
    text_list = sentence_batch['text']

    cleaned_text_list = []
    for text in text_list:
        # Convierte el texto a minúsculas
        text = text.lower()

        # Elimina URLs (usando la regex precompilada)
        text = URLS_RE.sub('', text)

        # Elimina las menciones @ y '#' de las redes sociales
        text = SOCIAL_RE.sub('', text)

        # Elimina los caracteres de puntuación
        text = strip_punctuation(text)

        # Elimina los números
        text = strip_numeric(text)

        # Elimina las palabras cortas
        text = strip_short(text,minsize=2)

        # Elimina las palabras comunes (stop words)
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in STOP_WORDS]

        cleaned_text_list.append(filtered_text)

    # Devuelve el texto limpio
    return {'text': cleaned_text_list}

In [39]:
sentences_corpus = subset.map(clean_text, batched=True)

Map: 100%|██████████| 1000000/1000000 [02:03<00:00, 8105.50 examples/s]


In [40]:
sentences_corpus['text'][:2]

[['lavado',
  'cerebro',
  'través',
  'medios',
  'comunicación',
  'amenaza',
  'fuerza',
  'través',
  'militares'],
 ['constante',
  'aluvión',
  'doble',
  'cañón',
  'requiriendo',
  'complicidad',
  'seres',
  'humanos',
  'reprimir',
  'engañar',
  'semejantes',
  'tan',
  'cacareada',
  'magia',
  'rápidamente',
  'desvanecería',
  'disiparía']]

## Carga y uso de modelo de embeddings Word2Vec

In [41]:
model = Word2Vec(sentences_corpus['text'], vector_size=100, window=5, min_count=2, workers=4, sg=1)

# Podemos guardar el modelo para uso futuro
model.save("models/word2vec.model")

In [42]:
model.wv['rey']

array([ 0.66221166,  0.6247485 , -0.5912262 ,  0.9801996 ,  0.29977125,
       -0.04997233,  0.13091032,  0.59405804, -0.02452532, -0.05881574,
        0.28410402, -0.31548807, -0.01153572,  0.59876513, -0.6800766 ,
        0.25985458, -0.4310782 , -0.62244195,  0.09897397, -0.57413894,
        0.5032274 ,  0.23944995,  0.55677557, -0.22253694,  0.12948108,
        0.8886712 , -0.2922969 , -0.51027125, -0.80960184,  0.0802108 ,
        0.22646986,  0.35475072, -0.02935858, -0.03748165, -0.3629772 ,
        0.43968788, -0.3353131 , -0.52782667, -0.08217013, -0.6342167 ,
        0.18188897, -0.78951514, -0.34181434,  0.15534294, -0.08824544,
        0.38322118,  0.01515038, -0.35002404, -0.24074869,  0.18641037,
       -0.02268551, -0.83950055, -0.10458446,  0.17566565, -0.3807993 ,
       -0.837186  , -0.29605255, -0.19258642,  0.26125848, -0.53568476,
        0.3221048 ,  0.27478924, -0.24105959,  0.37185273,  0.86435527,
        0.645737  , -0.7410816 , -0.22150946, -0.1640035 ,  0.01

In [43]:
##comida, ser, reina, television
model.wv.most_similar(['comida'], topn=3)

[('comer', 0.7892228960990906),
 ('refrigerios', 0.7405481338500977),
 ('sabrosa', 0.734772801399231)]

In [45]:
word_vectors = model.wv
vectors = word_vectors.vectors
words = word_vectors.index_to_key

## Almacenamiento de embeddings

In [46]:
df_vectors = pd.DataFrame(vectors)
df_vectors.to_csv('data/embeddings.tsv', sep='\t', index=False, header=False)

In [47]:
df_words = pd.DataFrame(words)
df_words.to_csv('data/labels.tsv', sep='\t', index=False, header=False)